In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.regularizers import l1_l2 


/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
train_data_dir = "/Users/deepanshudubb/Documents/accident/training"
val_data_dir = "/Users/deepanshudubb/Documents/accident/validation"

In [3]:
img_width, img_height = 150, 150

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary', 
    shuffle=False 
)

Found 1840 images belonging to 2 classes.


In [6]:
validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 460 images belonging to 2 classes.


In [7]:
def extract_features(generator):
    features = []
    labels = []
    for _ in range(generator.samples // generator.batch_size):
        batch_x, batch_y = next(generator)
        features.append(batch_x.reshape(batch_x.shape[0], -1)) 
        labels.append(batch_y)
    return np.concatenate(features), np.concatenate(labels)

In [8]:
train_features_knn, train_labels_knn = extract_features(train_generator)
val_features_knn, val_labels_knn = extract_features(validation_generator)

In [9]:
model_cnn = Sequential()

In [14]:
model_cnn = Sequential()


model_cnn.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001), input_shape=(img_width, img_height, 3)))
model_cnn.add(BatchNormalization()) 
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))


model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(1, activation='sigmoid'))

In [15]:
optimizer = Adam(learning_rate=0.0001) 
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [16]:
epochs = 20

model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)

Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 27s 449ms/step - accuracy: 0.6044 - loss: 63.3438 - val_accuracy: 0.5134 - val_loss: 51.5168
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.7500 - loss: 50.9404 - val_accuracy: 0.5134 - val_loss: 51.2831
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 30s 528ms/step - accuracy: 0.5801 - loss: 47.2106 - val_accuracy: 0.5134 - val_loss: 38.9080
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6875 - loss: 37.3764 - val_accuracy: 0.5134 - val_loss: 38.6528
Epoch 5/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 34s 608ms/step - accuracy: 0.6438 - loss: 34.4203 - val_accuracy: 0.5134 - val_loss: 29.4381
Epoch 6/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8438 - loss: 26.9595 - val_accuracy: 0.5134 - val_loss: 29.3212
Epoch 7/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 26s 454ms/step - accuracy: 0.6581 - loss: 25.1198 - val_accuracy: 0.5179 - val_loss: 21.9040
Epoch 8/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8438 - loss: 20.1747 - v

In [17]:
loss, accuracy = model_cnn.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)

print("CNN Accuracy:", accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.8286 - loss: 6.0443
CNN Accuracy: 0.6696428656578064
